# t-SNE projections of latent space.

In [1]:
import os, h5py
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from ipywidgets import ToggleButtons, VBox, Output
from pathlib import Path

# CONFIG — update if needed
RESULTS_DIR = Path("..") / "data" / "nucleus-ae"
TSNE_H5 = f"{RESULTS_DIR}/tsne_embeddings.h5"

# Enable interactive Matplotlib (ipympl) in notebooks
%matplotlib widget

plt.rcParams['figure.figsize'] = (7, 6)
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.bbox'] = 'tight'

print("[i] Matplotlib", matplotlib.__version__, "| Backend:", matplotlib.get_backend())
print("[i] Path:", TSNE_H5)


[i] Matplotlib 3.10.5 | Backend: widget
[i] Path: ..\data\nucleus-ae/tsne_embeddings.h5


In [2]:

def _safe_read(h5_path, dset_name):
    if not os.path.isfile(h5_path):
        return None
    with h5py.File(h5_path, "r") as h5:
        if dset_name in h5:
            return np.asarray(h5[dset_name][...], dtype=np.float32)
        return None

def load_tsne_embeddings():
    out = {}
    for k in ("tsne_cosine", "tsne_euclidean"):
        arr = _safe_read(TSNE_H5, k)
        if arr is not None:
            out[k.split("_",1)[1]] = arr
    return out

def _autosize_point_params(N):
    if N <= 50_000:
        return 6.0, 0.8
    elif N <= 200_000:
        return 2.5, 0.6
    elif N <= 1_000_000:
        return 0.8, 0.5
    else:
        return 0.4, 0.5

def scatter2d(ax, xy, title=None):
    N = xy.shape[0]
    s, alpha = _autosize_point_params(N)
    ax.scatter(xy[:,0], xy[:,1], s=s, alpha=alpha, rasterized=True)
    if title:
        ax.set_title(title)
    ax.set_xlabel("Dim 1"); ax.set_ylabel("Dim 2"); ax.grid(False)
    return ax

tsne_dict = load_tsne_embeddings()
print("[i] t-SNE keys:", list(tsne_dict.keys()))


[i] t-SNE keys: ['cosine', 'euclidean']


### Plot

In [3]:

#| label: ae1m-tsne

tsne_out = Output()
tsne_metric = ToggleButtons(
    options=[opt for opt in ("cosine", "euclidean") if opt in tsne_dict],
    description='Metric:',
)

def draw_tsne(metric):
    tsne_out.clear_output(wait=True)
    xy = tsne_dict.get(metric)
    if xy is None:
        with tsne_out:
            print(f"No t-SNE embedding for metric='{metric}'")
        return
    fig, ax = plt.subplots()
    scatter2d(ax, xy, title=f"t-SNE ({metric})")
    with tsne_out:
        plt.show()

def _on_tsne_change(change):
    if change['name'] == 'value':
        draw_tsne(change['new'])

if len(tsne_metric.options) > 0:
    draw_tsne(tsne_metric.options[0])

tsne_metric.observe(_on_tsne_change, names='value')
VBox([tsne_metric, tsne_out])
